In [67]:
import pandas as pd
from scipy import integrate
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display


neutron_pvs = ['RAD:Berthold:TotalNeutronRate', 'RAD:ELSE:Neutron', 'RAD:Thermo1:Neutron',\
    'RAD:Thermo2:Neutron', 'RAD:Thermo3:Neutron', 'RAD:Thermo4:Neutron', 'RAD:Thermo5:Neutron',\
        'RAD:Thermo6:Neutron', 'RAD:Thermo7:Neutron', 'RAD:Thermo8:Neutron', 'RAD:Thermo9:Neutron']

# Período com feixe
dt = 24 # horas
FROM_NOW = True
if FROM_NOW:
    final_date_b = datetime.now().strftime("%Y-%m-%d")
    initial_date_b = (datetime.now() - timedelta(hours=dt)).strftime("%Y-%m-%d")
    final_time_b = datetime.now().strftime('%H:%M:%S')
    initial_time_b = (datetime.now() - timedelta(hours=dt)).strftime('%H:%M:%S')

else:
    initial_date = '2022-01-22'
    initial_time = '12:00:00'
    final_date = '2022-01-23'
    final_time = '12:00:00'

# Período sem feixe
initial_date_nb = '2022-01-09'
initial_time_nb = '12:00:00'
final_date_nb = '2022-01-10'
final_time_nb = '12:00:00'

def encode_url(pvs: list, i_date: str, f_date: str, i_time: str, f_time: str):
    dfs = {}
    url_prefix = 'http://10.0.38.42/retrieval/data/getData.csv?pv='
    url_posfix = '&from={}T{}.000Z&to={}T{}.000Z'.format(i_date,\
        i_time.replace(':', '%3A'), f_date, f_time.replace(':', '%3A'))
    for _ in pvs:
        temp = pd.read_csv('{}{}{}'.format(url_prefix, _, url_posfix),\
            header = None, names = ["secs", "Dose Rate (uSv/h)", "severity", "status", "nanos"])
        temp['date'] = time = pd.to_datetime(temp['secs'], unit='s')
        temp['time (h)'] = (temp['secs'].subtract(temp['secs'][0]))/3600
        temp.drop('nanos', inplace=True, axis=1)
        temp.drop('severity', inplace=True, axis=1)
        temp.drop('status', inplace=True, axis=1)
        temp.drop('secs', inplace=True, axis=1)
        dfs[_] = temp
    return(dfs)

neutron_bgc = encode_url(neutron_pvs, initial_date_nb, final_date_nb, initial_time_nb, final_time_nb)
neutron_b = encode_url(neutron_pvs, initial_date_b, final_date_b, initial_time_b, final_time_b)

In [ ]:
# Dose total menos BG
dose_b = []
dose_bgc = []
dose_nbg = []
for _ in neutron_pvs:
    dose_b.append(integrate.trapz(neutron_b[_]['Dose Rate (uSv/h)'].values, neutron_b[_]['time (h)'].values))
    dose_bgc.append(integrate.trapz(neutron_bgc[_]['Dose Rate (uSv/h)'].values, neutron_bgc[_]['time (h)'].values))

for _ in range(len(dose_b)):
    dose_nbg.append(dose_b[_] - dose_bgc[_])
dic = {'Sonda': neutron_pvs, 'Dose sem BG': dose_nbg}
df = pd.DataFrame(dic, index=None)
display(df)

    


In [ ]:
# Média da taxa de dose de BG
neutron_bg_mean = []
for _ in neutron_bgc:
    neutron_bg_mean.append(np.mean(neutron_bgc[_]['Dose Rate (uSv/h)']))

# Média da taxa de dose com feixe menos o BG
neutron_b_mean = []
for _ in neutron_b:
    neutron_b_mean.append(np.mean(neutron_b[_]['Dose Rate (uSv/h)']))
for _ in range(len(neutron_b)):
    print(neutron_b_mean[_] - neutron_bg_mean[_])

In [ ]:
fig = plt.figure(figsize=(20,10))
axis = fig.add_subplot(111)
axis.set_title('Taxa de Dose {}'.format('RAD:Thermo9:Neutron'))
axis.set_xlabel('Tempo (s)')
axis.set_ylabel('Taxa de dose')
axis.legend(loc='best')
axis.plot(neutron_b['RAD:Thermo9:Neutron']['date'],\
    neutron_b['RAD:Thermo9:Neutron']['Dose Rate (uSv/h)'], label = 'RAD:Thermo9:Neutron')
axis.legend()

In [ ]:
fig = plt.figure(figsize=(20,10))
axis = fig.add_subplot(111)
axis.set_title('Taxa de Dose {}'.format('RAD:Thermo9:Neutron'))
axis.set_xlabel('Tempo (s)')
axis.set_ylabel('Taxa de dose')
axis.legend(loc='best')
axis.plot(neutron_b['RAD:Thermo9:Neutron']['time (h)'],\
    neutron_b['RAD:Thermo9:Neutron']['Dose Rate (uSv/h)'], label = 'RAD:Thermo9:Neutron Com Feixe')
axis.plot(neutron_bgc['RAD:Thermo9:Neutron']['time (h)'],\
    neutron_bgc['RAD:Thermo9:Neutron']['Dose Rate (uSv/h)'], label = 'RAD:Thermo9:Neutron Sem Feixe')
axis.legend()